In [ ]:
from typing import List, Mapping
import pickle

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from shadowbanning import get_graph, ShadowBanningRC, VANILLA_FILE_NAME, ERROR_FILE_NAME

In [ ]:
def reform_data(agents: List[ShadowBanningRC]):
    """This whole thing is jank lmao"""
    print_ = True
    full_rslts = []
    for agent in agents:
        x = agent.x
        k = len(x)
        ids = x[0].keys()
        ret = dict()
        for id in ids:
            vals = dict()
            y = np.zeros((2, k))
            z = np.zeros((2, k))
            for i in range(k):
                y[:,i] = x[i][id][0]
                z[:,i] = x[i][id][1]

            k = len(agent.gamma)
            gamma_y = np.zeros((k,))
            gamma_z = np.zeros((k,))
            for i in range(k):
                if agent.gamma[i][id] is None:
                    gamma_y[i] = np.nan
                    gamma_z[i] = np.nan
                else:
                    gamma_y[i] = agent.gamma[i][id][0].squeeze()
                    gamma_z[i] = agent.gamma[i][id][1].squeeze()
            vals["y"] = y.T[:,1]
            vals["z"] = z.T[:,1]
            vals["gamma_y"] = gamma_y
            vals["gamma_z"] = gamma_z
            ret[id] = vals
        full_rslts.append(ret)

    return full_rslts

In [ ]:
agents = None
with open(VANILLA_FILE_NAME, "rb") as f:
    agents = pickle.load(f)

data = reform_data(agents)

In [ ]:
def plot_experiment(data, fname_prefix):
    master_ratio = dict()
    for i, datum in enumerate(data):
        # Ratio data
        fig, axes = plt.subplots(ncols=2, figsize=(16,3))
        axes[0].set_xlabel("Iteration")
        axes[0].set_ylabel("Value")
        axes[1].set_xlabel("Iteration")
        axes[1].set_ylabel("Value")
        for id, val in datum.items():
            y = val["y"]
            z = val["z"]
            axes[0].plot(y, label=f"Agent {id} y")
            axes[0].plot(z, label=f"Agent {id} z")
            axes[1].plot(y/z, label=f"Agent {id} Ratio")
            axes[0].legend()
            axes[1].legend()
            axes[0].set_title("Y, Z")
            axes[1].set_title("Ratio")

            axes[0].set_xlabel("Iteration")
            axes[1].set_xlabel("Iteration")


            if(i+1 == id):
                print(f"Inserting master ratio for {id}")
                master_ratio[i] = y/z

        fig.suptitle(f"Agent {i+1}'s Perspective of Ratios")
        fig.savefig(f"{fname_prefix}_agent_{i}_ratios.png")

        # Gammas
        fig, ax = plt.subplots(figsize=(16,3))
        for id, val in datum.items():
            gamma_y = val["gamma_y"]
            gamma_z = val["gamma_z"]
            if gamma_y is None or gamma_z is None:
                continue
            ax.plot(gamma_y, label=f"Agent {id} gamma for Y")
            ax.plot(gamma_z, label=f"Agent {id} gamma for Z")
            ax.legend()
        fig.suptitle(f"Agent {i+1}'s Perspective of Gamma")
        fig.savefig(f"{fname_prefix}_agent_{i}_gammas.png")

    plt.figure(figsize=(16,8))
    plt.title("Local Agent Ratio Values")
    for i, datum in master_ratio.items():
        plt.plot(datum, label=f"Agent {i}")
    plt.legend()
    plt.xlabel("Iteration")
    plt.ylabel("Value")
    plt.savefig(f"{fname_prefix}_centralized_ratios.png")

def plot_pickle(fname, output_fname_prefix):
    with open(fname, "rb") as f:
        agents = pickle.load(f)
    data = reform_data(agents)
    plot_experiment(data, output_fname_prefix)

In [ ]:
plot_pickle(VANILLA_FILE_NAME, "vanilla")

In [ ]:
plot_pickle(ERROR_FILE_NAME, "error")

In [ ]:
graph = get_graph()
plt.figure()
nx.draw_kamada_kawai(graph, with_labels=True)
plt.title("Graph of Agent Communcations")
plt.savefig("graph.png")